In [9]:

import os.path
import pickle

from Ballot import Ballot
from CandidateModel import CandidateModel
from DefaultConfigOptions import *
from ElectionConstructor import ElectionConstructor, construct_irv, construct_h2h
from ModelStats import ModelStats
from NDPopulation import NDPopulation
from ProcessResult import ProcessResult
from Timings import Timings
from PluralityElection import PluralityElection


In [10]:
def create_model_and_population(ideology_bins: int, ideology_dim: int) -> (CandidateModel, NDPopulation):
    ideology_bins = 64
    hidden_ratio = 4
    n_hidden = hidden_ratio * ideology_bins * ideology_dim
    n_latent = ideology_bins * ideology_dim
    batch_size = 128
    learn_rate = .001

    model = CandidateModel(ideology_bins=ideology_bins,
                                     ideology_dim=ideology_dim,
                                     n_hidden=n_hidden,
                                     n_latent = n_latent,
                                     learn_rate= learn_rate)

    population_means = np.zeros(shape=(ideology_dim,))
    population_stddev = np.ones(shape=(ideology_dim,))
    pop = NDPopulation(population_means, population_stddev)
    return model, pop

In [11]:
def measure_representation(candidate: Candidate, voters: List[Voter]) -> float:
    n_voters = len(voters)
    balance = []
    for d in range(candidate.ideology.dim):
        lc = len([v for v in voters if v.ideology.vec[d] < candidate.ideology.vec[d]])
        balance.append(min(lc / n_voters, 1 - lc / n_voters))
    return float(np.mean(balance))

In [12]:
def gen_non_model_candidates(model: CandidateModel, population: NDPopulation) -> List[Candidate]:
    candidates: List[Candidate] = []
    if model.ready():
        if np.random.choice([True, False]):
            candidates += gen_pilot_candidates(population, .8)
        else:
            candidates += gen_random_candidates(population, 3)
    else:
        candidates += gen_pilot_candidates(population, .6)
        candidates += gen_random_candidates(population, 3)

    np.random.shuffle(candidates)
    return candidates

def gen_pilot_candidates(population: NDPopulation, spacing: float) -> List[Candidate]:
    candidates = []
    dim = population.dim
    d = spacing
    fuzz = .05
    c1_vec = np.random.normal(0, .01, dim)
    c1_vec[0] += np.random.normal(d, fuzz)
    candidates.append( Candidate("P-R", Independents, ideology=Ideology(c1_vec), quality=0))

    c2_vec = np.random.normal(0, .01, dim)
    c2_vec[0] -= np.random.normal(d, fuzz)
    candidates.append(Candidate("P-L", Independents, ideology=Ideology(c2_vec), quality=0))

    c3_vec = np.random.normal(0, .02, dim)
    candidates.append(Candidate("P-C", Independents, ideology=Ideology(c3_vec), quality=0))

    return candidates

def gen_random_candidates(population: NDPopulation, n: int)-> List[Candidate]:
    candidates = []
    for i in range(3):
        ivec = population.unit_sample_voter().ideology.vec * .5
        candidates.append(Candidate("r-" + str(i), Independents, Ideology(ivec), 0))

    return candidates



def run_sample_election(model: CandidateModel, process: ElectionConstructor, population: NDPopulation, train: bool):
    candidates = []
    model_entries = set(np.random.choice(range(6), 3, replace=False))
    r_candidates = gen_non_model_candidates(model, population)
    for i in range(6):
        if i in model_entries and model.ready():
            ideology = Ideology(model.choose_ideology(candidates))
            c = Candidate("m-" + str(i), Independents, ideology, 0)
        else:
            if train:
                c = r_candidates.pop()
            else:
                ideology = population.unit_sample_voter().ideology
                c = Candidate("r-" + str(i), Independents, ideology, 0)

        candidates += [c]

    voters = population.generate_unit_voters(1000)
    ballots = [Ballot(v, candidates, unit_election_config) for v in voters]
    result = process.run(ballots, set(candidates))
    winner = result.winner()
    balance = measure_representation(winner, voters)

    return winner, candidates, balance


In [13]:


def train_candidate_model(model: CandidateModel, process: ElectionConstructor, population: NDPopulation):
    timings = Timings()
    stats = ModelStats()
    first = True
    while model.global_step < 5000:
        winner, candidates, balance = run_sample_election(model, process, population, True)
        for i in range(len(candidates)):
            model.add_sample_from_candidates(candidates[i], candidates[0:i], winner)

        if model.ready():
            if first:
                print("starting to train")
                first = False

            stats.update(winner, candidates, balance)
            with timings.time_block("model.train"):
                model.train(128)
            s = model.global_step
            if (s < 100 and s % 10 == 0) or (s < 1000 and s % 100 == 0) or s % 1000 == 0:
                stats.print(process.name, model.global_step)
                if model.global_step < 10000:
                    stats.reset()

    timings.print()

In [14]:
def check_stats(stats: ModelStats, model: CandidateModel, process: ElectionConstructor, population: NDPopulation):
    results=[]
    timings = Timings()
    for i in range(1000):
        winner, candidates, balance = run_sample_election(model, process, population, train=False)
        stats.update(winner, candidates, balance)

In [15]:
def run_parameter_set(process: ElectionConstructor, ibins: int, dim: int):
    save_path = "models/cm-%s-%03d-%dD.p" % (process.name, ibins, dim)
    model, population = create_model_and_population(ibins, dim)
    if os.path.exists(save_path):
        with open(save_path, "rb") as f:
            model: CandidateModel = pickle.load(f)
    else:
        train_candidate_model(model, process, population)
        # Saving the model file is not working at this time.
        # model.save_to_file(save_path)

    stats = ModelStats()
    check_stats(stats, model, process, population)
    return stats, model


In [16]:
def review_model_candidates():
    ibins = 64
    for spacing in [.2, .3, .4, .5, .6, .7, .8, .9, 1.0]:
        for dim in [1,2,3,4]:
            names = {}
            model, population = create_model_and_population(ibins, dim)
            process = ElectionConstructor(constructor=construct_irv, name="Instant Runoff")
            stats = ModelStats()
            for i in range(100):

                candidates = gen_pilot_candidates(population, spacing)

                voters = population.generate_unit_voters(1000)
                ballots = [Ballot(v, candidates, unit_election_config) for v in voters]
                result = process.run(ballots, set(candidates))
                winner = result.winner()
                balance = measure_representation(winner, voters)
                stats.update(winner, candidates, balance)

                if not winner.name in names:
                    names[winner.name] = 0
                names[winner.name] += 1

                plurality = PluralityElection(ballots, set(candidates))
                r = plurality.result()

                # for c in candidates:
                #     print("[%s % 6.2f  %4d]" % (c.name, c.ideology.vec[0], r.vote_totals[c]), end= '')
                # print("d:% 6.2f %s " % (candidates[0].ideology.vec[0] - candidates[1].ideology.vec[0], winner.name))

            # stats.print("dim-%d" % dim, 0)
            print("dim-%d pilot spacing %5.2f" % (dim, spacing), end="")
            if "P-C" not in names:
                names["P-C"] = 0
            kk = list(names.keys())
            kk.sort(key=lambda k: k)
            for k in kk:
                print(" %s => %d" % (k, names[k]), end = "")
            print("")

review_model_candidates()

dim-1 pilot spacing  0.20 P-C => 0 P-L => 47 P-R => 53
dim-2 pilot spacing  0.20 P-C => 5 P-L => 42 P-R => 53
dim-3 pilot spacing  0.20 P-C => 10 P-L => 40 P-R => 50
dim-4 pilot spacing  0.20 P-C => 22 P-L => 38 P-R => 40
dim-1 pilot spacing  0.30 P-C => 0 P-L => 53 P-R => 47
dim-2 pilot spacing  0.30 P-C => 0 P-L => 46 P-R => 54
dim-3 pilot spacing  0.30 P-C => 2 P-L => 53 P-R => 45
dim-4 pilot spacing  0.30 P-C => 6 P-L => 53 P-R => 41
dim-1 pilot spacing  0.40 P-C => 0 P-L => 45 P-R => 55
dim-2 pilot spacing  0.40 P-C => 0 P-L => 45 P-R => 55
dim-3 pilot spacing  0.40 P-C => 0 P-L => 49 P-R => 51
dim-4 pilot spacing  0.40 P-C => 1 P-L => 50 P-R => 49
dim-1 pilot spacing  0.50 P-C => 0 P-L => 41 P-R => 59
dim-2 pilot spacing  0.50 P-C => 0 P-L => 47 P-R => 53
dim-3 pilot spacing  0.50 P-C => 0 P-L => 45 P-R => 55
dim-4 pilot spacing  0.50 P-C => 0 P-L => 52 P-R => 48
dim-1 pilot spacing  0.60 P-C => 0 P-L => 45 P-R => 55
dim-2 pilot spacing  0.60 P-C => 0 P-L => 50 P-R => 50
dim-3 pi

In [19]:
def train_models():
    dims = [4]
    processes = [
        ElectionConstructor(constructor=construct_irv, name="Instant Runoff"),
        ElectionConstructor(constructor=construct_h2h, name="Head-to-Head"),
    ]

    results = []
    for bins in [64, 128]:
        for process in processes:
            for dim in dims:
                stats, model = run_parameter_set(process, bins, dim)
                results.append(ProcessResult(process, bins, dim, stats))
                results[-1].print()

    for r in results:
        r.print()

train_models()

starting to train
 Instant Runoff     10,    10 random     12/     1  8.33% O:  0.57 model     48/     9 18.75% O:  0.53 chance of model_winner = 90.00%
 Instant Runoff     20,    10 random      6/     1 16.67% O:  0.40 model     54/     9 16.67% O:  0.54 chance of model_winner = 90.00%
 Instant Runoff     30,    10 random     21/     2  9.52% O:  0.42 model     39/     8 20.51% O:  0.44 chance of model_winner = 80.00%
 Instant Runoff     40,    10 random     12/     2 16.67% O:  0.65 model     48/     8 16.67% O:  0.59 chance of model_winner = 80.00%
 Instant Runoff     50,    10 random     18/     3 16.67% O:  0.59 model     42/     7 16.67% O:  0.58 chance of model_winner = 70.00%
 Instant Runoff     60,    10 random     12/     2 16.67% O:  0.48 model     48/     8 16.67% O:  0.58 chance of model_winner = 80.00%
 Instant Runoff     70,    10 random     18/     1  5.56% O:  0.47 model     42/     9 21.43% O:  0.56 chance of model_winner = 90.00%
 Instant Runoff     80,    10 random 

In [21]:
for i in range(10):
    if np.random.choice([True, False]):
        print("foo")
    else:
        print("bar")


bar
foo
bar
foo
bar
bar
bar
foo
bar
foo
